<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=91eb340c1ee090c32715b8fd266d9b8b098663afa19da08b5bde3f52d26e431f
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.0
    Uninstalling protobuf-6.32.0:
      Successfully uninstalled protobuf-6.32.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-08-28 15:01:35
-------------------
qualified stocks: 91
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  9.00 L
Current:  1.37 C
-------------------
Today PnL: -94.33 K (-0.69%)
Current PnL: -20.61 L (-13.86%)
CY Booked + Current PnL: -11.57 L (-7.78%)
-------------------
Total profit:  2.77 L
Total loss:  -23.38 L
-------------------
Total Booked + Current PnL: 15.41 L (12.66%)
Total Booked PnL: 36.02 L (29.6%)
Curr Year Booked PnL: 9.04 L (6.62%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.16 C
Est FTT PnL: 79.00 L (57.82%)
Deployed:  1.22 C
Current:  1.37 C
CAGR/XIRR %: 7.37%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,HINDALCO,756.01,-19.08,55.0,H-LC,7.46,112064.0,7752.0,8898.0,-0.75,7.43,7.94,15.96,9.0,0.87,0.88,25.58,X5K,ATH,METALS
53,MEDANTA,1486.00,14.41,63.0,H-MC,14.82,163328.0,37128.0,9048.0,0.57,29.42,5.54,36.59,108.0,4.10,1.28,45.67,XY24,NTT,HEALTHCARE
4,APOLLOHOSP,8285.00,-14.75,59.0,H-LC,8.26,170280.0,14103.0,11988.0,-0.69,9.03,7.04,16.71,22.0,1.18,1.33,28.04,X40N,BTT,HEALTHCARE
81,TTKPRESTIG,770.00,97.44,56.0,M-SC,8.41,85111.0,-15666.0,15763.0,1.49,-15.55,18.52,0.09,245.0,-0.99,0.67,11.82,OX40N,NTT,DURABLES
66,SBIN,863.00,-14.20,41.0,M-LC,4.18,212676.0,11196.0,16015.0,-0.66,5.56,7.53,13.51,60.0,0.70,1.67,18.85,XY25,NTT,BANKS


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,CAMS,5211.76,-9.59,35.0,H-SC,6.15,102760.0,756.0,43169.0,-1.51,0.74,42.01,43.06,122.0,0.02,0.81,17.98,X40N,ATH,MISC
59,RAJOOENG,143.33,-32.99,38.0,H-SC,9.57,101700.0,-800.0,41626.0,-1.67,-0.78,40.93,39.83,114.0,-0.02,0.80,0.69,AR,ATH,MISC


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,ABB,7934.00,-42.59,32.0,H-LC,13.16,195098.0,-21421.0,114327.0,-0.39,-9.89,58.60,42.91,7.0,-0.19,1.53,2.17,AR,NTT,ELECTRICAL
84,UNITDSPR,1644.00,-15.80,36.0,H-LC,8.76,169488.0,-12271.0,47524.0,-1.44,-6.75,28.04,19.39,86.0,-0.26,1.33,0.85,X40N,NTT,BREWERIES
47,JIOFIN,387.00,-1.72,38.0,H-LC,15.37,217315.0,-91.0,53590.0,-1.68,-0.04,24.66,24.61,48.0,-0.00,1.70,54.75,XY24,BTT,FINANCE
33,HCLTECH,1943.91,-0.22,38.0,H-LC,10.38,177071.0,-13575.0,60080.0,-2.77,-7.12,33.93,24.40,8.0,-0.23,1.39,7.68,X40,ATH,IT
45,ITC,452.00,-38.77,40.0,H-LC,3.05,146819.0,-3113.0,17706.0,1.00,-2.08,12.06,9.74,4.0,-0.18,1.15,4.03,X40,NTT,FMCG


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
45,ITC,452.0,-38.77,40.0,H-LC,3.05,146819.0,-3113.0,17706.0,1.00,-2.08,12.06,9.74,4.0,-0.18,1.15,4.03,X40,NTT,FMCG
51,LTIM,7230.2,-6.94,48.0,H-LC,3.09,184554.0,-15756.0,75741.0,-0.76,-7.87,41.04,29.94,16.0,-0.21,1.45,27.50,X200,ATH,IT
55,NESTLEIND,1377.0,-11.53,53.0,H-LC,3.17,272376.0,6950.0,49845.0,0.15,2.62,18.30,21.40,11.0,0.14,2.13,10.61,XY25,NTT,FMCG
20,CIPLA,1795.0,-18.75,61.0,H-LC,4.24,216666.0,12166.0,29250.0,0.11,5.95,13.50,20.25,10.0,0.42,1.70,15.20,X40N,BTT,PHARMA
5,ASIANPAINT,4250.0,-12.93,54.0,H-LC,6.71,169762.0,-31624.0,119241.0,0.61,-15.70,70.24,43.51,27.0,-0.27,1.33,18.17,X40,BTT,PAINTS


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
37,HONAUT,58357.33,-17.33,57.0,H-SC,3.19,116430.0,-11508.0,58646.0,2.43,-9.00,50.37,36.84,143.0,-0.20,0.91,19.20,X40N,ATH,ELECTRICAL
85,VAIBHAVGBL,521.00,37.74,38.0,H-SC,11.20,91860.0,-60131.0,134253.0,1.90,-39.56,146.15,48.77,125.0,-0.45,0.72,6.92,XR,NTT,JEWELLERY
48,KANSAINER,340.00,-68.89,48.0,H-SC,3.35,214371.0,-55296.0,91622.0,1.76,-20.51,42.74,13.47,138.0,-0.60,1.68,8.43,XY24,NTT,PAINTS
13,BERGEPAINT,680.00,-19.60,42.0,H-MC,8.51,216068.0,-11271.0,59332.0,1.60,-4.96,27.46,21.14,106.0,-0.19,1.69,21.79,XY24,NTT,PAINTS
74,SYMPHONY,1306.00,-41.90,31.0,M-SC,17.02,123926.0,-47215.0,47154.0,1.59,-27.59,38.05,-0.03,196.0,-1.00,0.97,1.59,OX40N,NTT,DURABLES


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
69,SIEMENS,7969.85,-5.18,42.0,H-LC,33.21,76200.0,-33972.0,123048.0,-3.68,-30.84,161.48,80.85,15.0,-0.28,0.60,12.23,AR,ATH,ELECTRICAL
61,REPCOHOME,880.00,-62.74,23.0,H-SC,7.23,138265.0,-66733.0,211974.0,-2.91,-32.55,153.31,70.85,134.0,-0.31,1.08,13.59,XY24,NTT,FINANCE
67,SFL,1287.00,36.81,55.0,M-SC,10.04,180561.0,-81657.0,151491.0,-2.89,-31.14,83.90,26.63,229.0,-0.54,1.41,12.26,XY24,NTT,MISC
40,INDIAMART,4850.92,-51.09,49.0,H-SC,12.42,136682.0,13346.0,120417.0,-2.85,10.82,88.10,108.45,119.0,0.11,1.07,35.59,AR,ATH,MISC
79,TITAGARH,1548.00,-9.77,47.0,H-SC,9.56,147796.0,-44219.0,123099.0,-2.85,-23.03,83.29,41.08,158.0,-0.36,1.16,25.09,XY24,NTT,ENGINEERING


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,TTKPRESTIG,770.0,97.44,56.0,M-SC,8.41,85111.0,-15666.0,15763.0,1.49,-15.55,18.52,0.09,245.0,-0.99,0.67,11.82,OX40N,NTT,DURABLES
82,UJJIVANSFB,60.0,82.03,40.0,H-SC,15.18,114750.0,-27729.0,47254.0,-0.68,-19.46,41.18,13.70,163.0,-0.59,0.90,35.44,OX40N,NTT,BANKS
48,KANSAINER,340.0,-68.89,48.0,H-SC,3.35,214371.0,-55296.0,91622.0,1.76,-20.51,42.74,13.47,138.0,-0.60,1.68,8.43,XY24,NTT,PAINTS
41,INDIGOPNTS,1408.0,107.59,39.0,M-SC,2.01,136251.0,-38308.0,38341.0,-0.45,-21.95,28.14,0.02,221.0,-1.00,1.07,18.52,OX40N,NTT,PAINTS
19,CERA,9475.0,-21.48,37.0,H-SC,3.02,112140.0,-31663.0,58414.0,-1.31,-22.02,52.09,18.60,149.0,-0.54,0.88,23.75,OX40N,NTT,CERAMICS


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TCS,4998.00,-26.76,48.0,H-LC,15.76,241714.0,-54220.0,148122.0,-1.85,-18.32,61.28,31.73,1.0,-0.37,1.89,3.19,X40,BTT,IT
43,INFY,2275.00,-18.16,50.0,H-LC,11.83,268500.0,6899.0,138734.0,-1.94,2.64,51.67,55.67,3.0,0.05,2.10,8.90,X40,BTT,IT
45,ITC,452.00,-38.77,40.0,H-LC,3.05,146819.0,-3113.0,17706.0,1.00,-2.08,12.06,9.74,4.0,-0.18,1.15,4.03,X40,NTT,FMCG
87,VBL,671.64,-9.95,54.0,H-LC,8.65,273600.0,8560.0,87744.0,-1.44,3.23,32.07,36.33,5.0,0.10,2.14,16.96,X40N,ATH,FMCG
1,ABB,7934.00,-42.59,32.0,H-LC,13.16,195098.0,-21421.0,114327.0,-0.39,-9.89,58.60,42.91,7.0,-0.19,1.53,2.17,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,MEDANTA,1486.0,14.41,63.0,H-MC,14.82,163328.0,37128.0,9048.0,0.57,29.42,5.54,36.59,108.0,4.10,1.28,45.67,XY24,NTT,HEALTHCARE
27,DMART,5201.0,-7.76,72.0,H-LC,11.92,230241.0,45760.0,24613.0,-0.37,24.80,10.69,38.14,38.0,1.86,1.80,38.04,X40N,NTT,FMCG


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,MEDANTA,1486.00,14.41,63.0,H-MC,14.82,163328.0,37128.0,9048.0,0.57,29.42,5.54,36.59,108.0,4.10,1.28,45.67,XY24,NTT,HEALTHCARE
14,BLUESTARCO,2326.38,8.10,56.0,H-MC,4.82,186500.0,21830.0,46140.0,-2.07,13.26,24.74,41.28,89.0,0.47,1.46,22.30,X40N,ATH,AC
88,VOLTAS,1918.49,-2.90,56.0,H-MC,6.54,203820.0,12078.0,83953.0,0.85,6.30,41.19,50.08,99.0,0.14,1.60,13.52,XY25,ATH,AC
34,HEROMOTOCO,5949.07,-3.95,70.0,H-MC,14.91,178367.0,27454.0,29859.0,0.45,18.19,16.74,37.97,93.0,0.92,1.40,48.10,AR,ATH,AUTO
24,DABUR,735.00,-7.62,51.0,H-MC,3.81,201142.0,4114.0,85505.0,-1.27,2.09,42.51,45.49,102.0,0.05,1.58,15.40,XY24,BTT,FMCG


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
34,HEROMOTOCO,5949.07,-3.95,70.0,H-MC,14.91,178367.0,27454.0,29859.0,0.45,18.19,16.74,37.97,93.0,0.92,1.40,48.10,AR,ATH,AUTO
40,INDIAMART,4850.92,-51.09,49.0,H-SC,12.42,136682.0,13346.0,120417.0,-2.85,10.82,88.10,108.45,119.0,0.11,1.07,35.59,AR,ATH,MISC
89,WHIRLPOOL,2270.00,-42.18,48.0,M-SC,5.61,97665.0,6166.0,72585.0,1.34,6.74,74.32,86.07,223.0,0.08,0.77,41.28,XR,NTT,DURABLES
90,WIPRO,420.00,-11.52,53.0,M-LC,7.77,155744.0,4799.0,104660.0,-0.33,3.18,67.20,72.51,53.0,0.05,1.22,9.39,XR,NTT,IT
59,RAJOOENG,143.33,-32.99,38.0,H-SC,9.57,101700.0,-800.0,41626.0,-1.67,-0.78,40.93,39.83,114.0,-0.02,0.80,0.69,AR,ATH,MISC


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.00,-43.66,63.0,L-SC,28.07,81907.0,-31642.0,71693.0,-0.70,-27.87,87.53,35.27,268.0,-0.44,0.64,100.47,XR,NTT,HOTELS
3,ALKYLAMINE,4546.37,-6.19,37.0,H-SC,16.30,87238.0,-13725.0,108254.0,-1.88,-13.59,124.09,93.63,148.0,-0.13,0.68,29.94,SR,ATH,CHEMICALS
52,MASFIN,398.61,-18.25,49.0,H-SC,12.63,92700.0,-5280.0,26883.0,0.54,-5.39,29.00,22.05,152.0,-0.20,0.73,34.75,XR,ATH,FINANCE
35,HINDALCO,756.01,-19.08,55.0,H-LC,7.46,112064.0,7752.0,8898.0,-0.75,7.43,7.94,15.96,9.0,0.87,0.88,25.58,X5K,ATH,METALS
82,UJJIVANSFB,60.00,82.03,40.0,H-SC,15.18,114750.0,-27729.0,47254.0,-0.68,-19.46,41.18,13.70,163.0,-0.59,0.90,35.44,OX40N,NTT,BANKS


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.00,-43.66,63.0,L-SC,28.07,81907.0,-31642.0,71693.0,-0.70,-27.87,87.53,35.27,268.0,-0.44,0.64,100.47,XR,NTT,HOTELS
27,DMART,5201.00,-7.76,72.0,H-LC,11.92,230241.0,45760.0,24613.0,-0.37,24.80,10.69,38.14,38.0,1.86,1.80,38.04,X40N,NTT,FMCG
34,HEROMOTOCO,5949.07,-3.95,70.0,H-MC,14.91,178367.0,27454.0,29859.0,0.45,18.19,16.74,37.97,93.0,0.92,1.40,48.10,AR,ATH,AUTO
14,BLUESTARCO,2326.38,8.10,56.0,H-MC,4.82,186500.0,21830.0,46140.0,-2.07,13.26,24.74,41.28,89.0,0.47,1.46,22.30,X40N,ATH,AC
67,SFL,1287.00,36.81,55.0,M-SC,10.04,180561.0,-81657.0,151491.0,-2.89,-31.14,83.90,26.63,229.0,-0.54,1.41,12.26,XY24,NTT,MISC


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.93
1,25,42.04
2,50,72.20


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    38.33
LC    34.53
MC    27.16
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     29.16
X40N     14.42
X40      14.14
XY25     10.04
XR        9.29
AR        8.74
OX40N     7.61
X5K       2.32
MH        1.73
X200      1.45
SR        1.12
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    26.06
H-MC    24.20
H-SC    23.14
M-SC    13.61
M-LC     7.38
M-MC     2.63
L-SC     1.58
L-LC     1.09
L-MC     0.33
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.94,-1.68,35.33
IT,12.40,-18.04,76.99
BANKS,7.62,-17.10,66.82
FINANCE,7.47,-26.53,73.18
MISC,6.06,-18.93,87.10
PAINTS,5.77,-18.53,41.89
ELECTRICAL,4.90,-12.40,61.07
AUTO,4.73,-12.07,60.83
AC,3.82,0.61,33.05


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2913102.0,23
XR,1107389.0,13
AR,1095549.0,10
X40,796832.0,10
X40N,525859.0,11
OX40N,488117.0,10
XY25,460463.0,7
SR,199626.0,2
X5K,142604.0,2


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2823566.0,27
M-SC,1510642.0,17
H-MC,1289898.0,18
H-LC,1133743.0,17
M-LC,436476.0,5
M-MC,341437.0,2
L-SC,258801.0,3
L-MC,60042.0,1
L-LC,45757.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      955472.0      6
M-SC       XY24      885421.0      7
H-SC       AR        667849.0      5
           XR        531982.0      6
H-MC       XY24      495087.0      5
H-LC       X40       483883.0      5
M-MC       XY24      341437.0      2
H-LC       AR        237375.0      2
H-MC       X40       228132.0      4
H-SC       OX40N     211874.0      4
H-LC       X40N      201119.0      5
H-MC       XY25      201011.0      2
H-SC       SR        199626.0      2
           X40N      194783.0      3
M-LC       XY24      182095.0      2
L-SC       XR        168101.0      2
M-SC       AR        160466.0      2
M-LC       X5K       133706.0      1
M-SC       OX40N     132172.0      4
H-MC       X40N      129957.0      3
M-SC       XY25      124543.0      1
           XR        123223.0      2
H-MC       XR        119381.0      1
M-LC       XR        104660.0      1
L-SC       OX40N      90700.0      1
M-SC       X40        84817.0      1
H-LC       X200       75741.0      1
           XY25       73137.0      2
H-SC       MH         61980.0      1
L-MC       XR         60042.0      1
H-LC       XY24       53590.0      1
H-MC       OX40N      53371.0      1
L-LC       XY25       45757.0      1
H-MC       MH         33100.0      1
           AR         29859.0      1
M-LC       XY25       16015.0      1
H-LC       X5K         8898.0      1

In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 54.0 seconds
